In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

In [5]:
from TM1py import TM1Service

In [28]:
with TM1Service(address="xxx.xxx.xxx.xxx", port=16333, user="admin", password="apple", ssl=True) as tm1:
    data = tm1.power_bi.execute_view(cube_name="Revenue", view_name="h2o_gross_revenue_prediction", private=False)

In [16]:
from sklearn import preprocessing

In [20]:
le = preprocessing.LabelEncoder()
le.fit(data["Year"])
data["Year"] = le.transform(data["Year"])

In [21]:
le = preprocessing.LabelEncoder()
le.fit(data["Month"])
data["Month"] = le.transform(data["Month"])

In [29]:
data

,product,Channel,organization,Year,Month,Unit Price,Units Sold,Unit Cost
0,21002,10,101,Y1,Jan,70.0,271.94,47.043099999999995
1,21002,10,101,Y1,Feb,70.0,271.94,47.043099999999995
2,21002,10,101,Y1,Mar,70.0,271.94,47.043099999999995
3,21002,10,101,Y1,Apr,70.0,271.94,47.043099999999995
4,21002,10,101,Y1,May,70.0,271.94,47.043099999999995
...,...,...,...,...,...,...,...,...
21613,42003,30,403,Y3,Aug,320.0,13.0,0.0
21614,42003,30,403,Y3,Sep,320.0,13.0,0.0
21615,42003,30,403,Y3,Oct,320.0,13.0,0.0
21616,42003,30,403,Y3,Nov,320.0,13.0,0.0


In [30]:
train_size = int(data.shape[0] * 0.9)
data[:train_size].to_csv('train.csv', index=False)
data[train_size:].to_csv('eval.csv', index=False)
train_file_path = 'train.csv'
test_file_path = 'eval.csv'

In [31]:
train_size

19456

In [32]:
# Initialize the structured data regressor.
reg = ak.StructuredDataRegressor(
    overwrite=True,
    max_trials=4) # It tries 10 different models.
# Feed the structured data regressor with training data.
reg.fit(
    # The path to the train.csv file.
    train_file_path,
    # The name of the label column.
    'Units Sold',
    epochs=20)

Trial 4 Complete [00h 00m 12s]
val_loss: 528.6915893554688

Best val_loss So Far: 528.6915893554688
Total elapsed time: 00h 00m 45s
INFO:tensorflow:Oracle triggered exit
Epoch 1/20
608/608 [==============================] - 1s 1ms/step - loss: 3056.1045 - mean_squared_error: 3056.1045
Epoch 2/20
608/608 [==============================] - 1s 1ms/step - loss: 2566.6926 - mean_squared_error: 2566.6926
Epoch 3/20
608/608 [==============================] - 1s 1ms/step - loss: 2522.0537 - mean_squared_error: 2522.0537
Epoch 4/20
608/608 [==============================] - 1s 1ms/step - loss: 2501.5632 - mean_squared_error: 2501.5632
Epoch 5/20
608/608 [==============================] - 1s 1ms/step - loss: 2494.4998 - mean_squared_error: 2494.4998
Epoch 6/20
608/608 [==============================] - 1s 1ms/step - loss: 2487.9465 - mean_squared_error: 2487.9465
Epoch 7/20
608/608 [==============================] - 1s 1ms/step - loss: 2484.4280 - mean_squared_error: 2484.4280
Epoch 8/20
608/608

In [33]:
# Predict with the best model.
predicted_y = reg.predict(test_file_path)

In [34]:
# Evaluate the best model with testing data.
print(reg.evaluate(test_file_path, 'Units Sold'))

68/68 [==============================] - 0s 652us/step - loss: 621.9655 - mean_squared_error: 621.9655
[621.9654541015625, 621.9654541015625]


In [14]:
predicted_y

array([[13.016721],
       [13.972049],
       [28.356125],
       ...,
       [19.356752],
       [19.5629  ],
       [19.689722]], dtype=float32)